## billboard 200 cleaning

In [18]:
import pandas as pd
import os
from termcolor import colored
from IPython.display import display, HTML

# define path
data_dir = '../data/ranking_5'
# Exclude list
exclude_files = ['hot100.csv', 'charts.csv']

# 2. 获取目录下所有符合条件的 CSV 文件
# 过滤逻辑：后缀是 .csv 且 不在排除名单中
other_csv_files = [
    f for f in os.listdir(data_dir)
    if f.endswith('.csv') and f not in exclude_files
]

# Dictionary to store dataframes: { 'filename': dataframe }
dfs = {}

print(colored(f"Starting batch import from {data_dir}...", 'blue'))

for file_name in other_csv_files:
    file_path = os.path.join(data_dir, file_name)
    try:
        # 动态创建变量名或存入字典
        # 使用文件名（去掉.csv）作为 key
        df_name = file_name.replace('.csv', '')
        dfs[df_name] = pd.read_csv(file_path, dtype={4: str})

        print(colored(f"Successfully imported: {file_name} | Shape: {dfs[df_name].shape}", 'green'))
    except Exception as e:
        print(colored(f"Failed to import {file_name}: {e}", 'red'))

# 4. 验证导入结果
print("-" * 30)
print(f"Total files imported: {len(dfs)}")
print(f"Available keys: {list(dfs.keys())}")

# View shape statistics for all imported files
print(colored("\n" + "=" * 40, 'blue'))
print(colored("Summary of Imported Datasets:", 'blue', attrs=['bold']))
print(colored("=" * 40, 'blue'))

# Iterate through each dataframe in the dictionary
for name, df in dfs.items():
    # 使用 f-string 格式化输出，让结果对齐
    print(f"Dataset: {name:<20} | Rows: {df.shape[0]:>8,} | Columns: {df.shape[1]:>2}")

print(colored("=" * 40, 'blue'))

Starting batch import from ../data/ranking_5...
Successfully imported: digital_songs.csv | Shape: (52175, 8)
Successfully imported: billboard200.csv | Shape: (639746, 8)
Successfully imported: streaming_songs.csv | Shape: (33300, 8)
Successfully imported: radio.csv | Shape: (91300, 8)
------------------------------
Total files imported: 4
Available keys: ['digital_songs', 'billboard200', 'streaming_songs', 'radio']

Summary of Imported Datasets:
Dataset: digital_songs        | Rows:   52,175 | Columns:  8
Dataset: billboard200         | Rows:  639,746 | Columns:  8
Dataset: streaming_songs      | Rows:   33,300 | Columns:  8
Dataset: radio                | Rows:   91,300 | Columns:  8


In [19]:
new_columns = ['date', 'song', 'artist', 'rank', 'last_week', 'peak_rank', 'weeks_on_board']

for name, df in dfs.items():
    print(f"Cleaning: {name}")

    # 1. 统一列数 (Ensuring consistent column count)
    if 'Image URL' in df.columns:
        df.drop(columns=['Image URL'], inplace=True)

    # 2. 统一列名 (Standardizing column names)
    if len(df.columns) == len(new_columns):
        df.columns = new_columns
    else:
        print(colored(f"Warning: {name} structure mismatch!", 'red'))
        continue

        # 3. 转换日期格式 (IMPORTANT: Date format conversion)
    # errors='coerce' 会将无法解析的日期变为 NaT
    df['date'] = pd.to_datetime(df['date'], errors='coerce')

    # 4. 文本清洗 (Text cleaning)
    df['song'] = df['song'].astype(str).str.strip().str.lower()
    df['artist'] = df['artist'].astype(str).str.strip().str.lower()

    # 5. 数值列转换 (Numeric conversion)
    cols_to_fix = ['rank', 'last_week', 'peak_rank', 'weeks_on_board']
    for col in cols_to_fix:
        df[col] = pd.to_numeric(df[col], errors='coerce')

print(colored("Success! Date and other columns are all processed.", 'green'))

Cleaning: digital_songs
Cleaning: billboard200
Cleaning: streaming_songs
Cleaning: radio
Success! Date and other columns are all processed.


In [20]:
# 遍历字典中的每一个 DataFrame
for name, df in dfs.items():
    # 打印彩色标题以区分不同的表
    print("\n" + "=" * 50)
    print(colored(f" DATASET: {name.upper()} ", 'white', 'on_blue', attrs=['bold']))
    print("=" * 50)

    # 1. 输出 Info (由于 info 默认直接 print，我们手动加说明)
    print(colored("\n[ 1. Basic Information ]", 'blue', attrs=['bold']))
    df.info()

    # 2. 输出 Describe (使用 display 让 DataFrame 在 Notebook 里以表格形式美化呈现)
    print(colored("\n[ 2. Statistical Summary ]", 'blue', attrs=['bold']))
    # T 是转置的意思，如果列很多，转置后看统计指标会更舒服
    display(df.describe().T)

    print("\n")


 DATASET: DIGITAL_SONGS 

[ 1. Basic Information ]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52175 entries, 0 to 52174
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            52175 non-null  datetime64[ns]
 1   song            52175 non-null  object        
 2   artist          52175 non-null  object        
 3   rank            52175 non-null  int64         
 4   last_week       52171 non-null  float64       
 5   peak_rank       52175 non-null  int64         
 6   weeks_on_board  42682 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 2.8+ MB

[ 2. Statistical Summary ]


,count,mean,min,25%,50%,75%,max,std
date,52175,2014-11-01 23:52:24.609487360,2004-10-27 00:00:00,2009-10-21 00:00:00,2014-10-22 00:00:00,2019-10-23 00:00:00,2025-10-22 00:00:00,NaN
rank,52175.0,24.871107,1.0,12.0,24.0,37.0,50.0,14.417298
last_week,52171.0,23.253877,1.0,10.0,22.0,35.0,75.0,14.964232
peak_rank,52175.0,10.426909,1.0,1.0,5.0,16.0,114.0,11.997793
weeks_on_board,42682.0,14.325711,1.0,6.0,11.0,19.0,113.0,11.861434





 DATASET: BILLBOARD200 

[ 1. Basic Information ]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 639746 entries, 0 to 639745
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   date            639746 non-null  datetime64[ns]
 1   song            639746 non-null  object        
 2   artist          639746 non-null  object        
 3   rank            639746 non-null  int64         
 4   last_week       639738 non-null  float64       
 5   peak_rank       639746 non-null  int64         
 6   weeks_on_board  582734 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 34.2+ MB

[ 2. Statistical Summary ]


,count,mean,min,25%,50%,75%,max,std
date,639746,1995-02-22 07:34:52.036527104,1963-08-14 00:00:00,1979-10-31 00:00:00,1995-03-01 00:00:00,2010-06-30 00:00:00,2025-10-22 00:00:00,NaN
rank,639746.0,99.355708,1.0,50.0,99.0,148.0,200.0,57.381735
last_week,639738.0,92.267305,1.0,43.0,91.0,139.0,200.0,56.347736
peak_rank,639746.0,35.430129,1.0,2.0,14.0,52.0,991.0,47.399443
weeks_on_board,582734.0,43.510871,1.0,7.0,17.0,42.0,988.0,82.25624





 DATASET: STREAMING_SONGS 

[ 1. Basic Information ]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33300 entries, 0 to 33299
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            33300 non-null  datetime64[ns]
 1   song            33300 non-null  object        
 2   artist          33300 non-null  object        
 3   rank            33300 non-null  int64         
 4   last_week       33300 non-null  int64         
 5   peak_rank       33300 non-null  int64         
 6   weeks_on_board  27768 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(3), object(2)
memory usage: 1.8+ MB

[ 2. Statistical Summary ]


,count,mean,min,25%,50%,75%,max,std
date,33300,2019-06-08 11:59:59.999999744,2013-01-23 00:00:00,2016-03-30 00:00:00,2019-06-08 12:00:00,2022-08-17 00:00:00,2025-10-22 00:00:00,NaN
rank,33300.0,25.5,1.0,13.0,25.5,38.0,50.0,14.431086
last_week,33300.0,23.182553,1.0,11.0,22.0,35.0,50.0,14.033142
peak_rank,33300.0,10.16006,1.0,1.0,5.0,15.0,148.0,11.95033
weeks_on_board,27768.0,17.505186,2.0,6.0,12.0,23.0,153.0,17.019799





 DATASET: RADIO 

[ 1. Basic Information ]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91300 entries, 0 to 91299
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            91300 non-null  datetime64[ns]
 1   song            91300 non-null  object        
 2   artist          91300 non-null  object        
 3   rank            91300 non-null  int64         
 4   last_week       91300 non-null  int64         
 5   peak_rank       91300 non-null  int64         
 6   weeks_on_board  88725 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(3), object(2)
memory usage: 4.9+ MB

[ 2. Statistical Summary ]


,count,mean,min,25%,50%,75%,max,std
date,91300,2008-04-26 12:00:00,1990-10-31 00:00:00,1999-07-28 00:00:00,2008-04-26 12:00:00,2017-01-25 00:00:00,2025-10-22 00:00:00,NaN
rank,91300.0,25.5,1.0,13.0,25.5,38.0,50.0,14.430949
last_week,91300.0,26.178653,1.0,13.0,26.0,39.0,89.0,15.42275
peak_rank,91300.0,18.805531,1.0,5.0,16.0,30.0,61.0,14.537344
weeks_on_board,88725.0,13.42634,2.0,7.0,11.0,17.0,95.0,9.536824


In [21]:
# 统一统计并列出所有文件的缺失值 (Consolidated Missing Value Audit)
print(colored("=" * 50, 'blue', attrs=['bold']))
print(colored("MISSING VALUES SUMMARY BY DATASET", 'blue', attrs=['bold']))
print(colored("=" * 50, 'blue'))

for name, df in dfs.items():
    # 计算每列的缺失值总数
    null_counts = df.isnull().sum()

    # 只筛选出存在缺失值的列 (Optional: only show columns with nulls)
    missing_data = null_counts[null_counts > 0]

    print(f"\n📂 Dataset: {colored(name.upper(), 'yellow', attrs=['bold'])}")
    print(f"Total Rows: {len(df):,}")

    if not missing_data.empty:
        # 列出每个有缺失值的字段及其数量
        for col, count in missing_data.items():
            percentage = (count / len(df)) * 100
            print(f"  - {col:<15} : {count:>8,} missing ({percentage:>6.2f}%)")
    else:
        print(colored("  ✅ No missing values found!", 'green'))

print(colored("\n" + "=" * 50, 'blue'))

MISSING VALUES SUMMARY BY DATASET

📂 Dataset: DIGITAL_SONGS
Total Rows: 52,175
  - last_week       :        4 missing (  0.01%)
  - weeks_on_board  :    9,493 missing ( 18.19%)

📂 Dataset: BILLBOARD200
Total Rows: 639,746
  - last_week       :        8 missing (  0.00%)
  - weeks_on_board  :   57,012 missing (  8.91%)

📂 Dataset: STREAMING_SONGS
Total Rows: 33,300
  - weeks_on_board  :    5,532 missing ( 16.61%)

📂 Dataset: RADIO
Total Rows: 91,300
  - weeks_on_board  :    2,575 missing (  2.82%)



In [22]:
# 1. 获取当前 notebook 文件的绝对路径 (Get absolute path of current notebook)
current_dir = os.path.dirname(os.path.abspath('__file__'))

# 2. 定位到项目根目录 (Locate project root)
project_root = os.path.abspath(os.path.join(current_dir, ".."))

# 3. 定义输出目录路径 (Define output directory path)
output_dir = os.path.join(project_root, 'output', 'batch_cleaned')

# 4. 如果目录不存在则创建 (Create directory if not exists)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 5. 循环导出字典中的每个 DataFrame (Iterate and export from the dictionary)
for name, df in dfs.items():
    # 使用绝对路径拼接文件名 (Construct file path using absolute positioning)
    file_name = f"{name}_cleaned.csv"
    file_path = os.path.join(output_dir, file_name)

    # 执行导出 (Execute export)
    df.to_csv(file_path, index=False, encoding='utf-8-sig')

    print(colored(f"File strictly saved to: {file_path}", 'green'))

print(colored("\nAll batch files have been successfully exported!", 'blue', attrs=['bold']))

File strictly saved to: /Users/jesse/Desktop/DA_prj1_Music/output/batch_cleaned/digital_songs_cleaned.csv
File strictly saved to: /Users/jesse/Desktop/DA_prj1_Music/output/batch_cleaned/billboard200_cleaned.csv
File strictly saved to: /Users/jesse/Desktop/DA_prj1_Music/output/batch_cleaned/streaming_songs_cleaned.csv
File strictly saved to: /Users/jesse/Desktop/DA_prj1_Music/output/batch_cleaned/radio_cleaned.csv

All batch files have been successfully exported!
